<a href="https://colab.research.google.com/github/GenAk95/GenAI_Projects/blob/main/Yuotube_Text_Summary_GenAI_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
pip install gradio

In [7]:
pip install youtube_transcript_api

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.9/485.9 kB 16.3 MB/s eta 0:00:00


In [9]:
pip install transformers

In [10]:
pip install nltk

example urls- :

1.   https://youtu.be/AO6WbXTeDow



In [15]:
from youtube_transcript_api import YouTubeTranscriptApi
from youtube_transcript_api._errors import TranscriptsDisabled, NoTranscriptFound
from urllib.parse import urlparse, parse_qs
import gradio as gr
from transformers import pipeline
import nltk
from nltk.tokenize import PunktSentenceTokenizer

# Download and initialize the Punkt tokenizer
nltk.download('punkt')
punkt_tokenizer = PunktSentenceTokenizer()

def sent_tokenize(text):
    return punkt_tokenizer.tokenize(text)

# Load the summarization model pipeline
text_summary = pipeline("summarization", model="philschmid/distilbart-cnn-12-6-samsum")

# Function to summarize long text by breaking into smaller chunks
def summarize_text(full_text, max_chunk_words=500):
    sentences = sent_tokenize(full_text)
    chunks = []
    current_chunk = ""

    for sentence in sentences:
        if len(current_chunk.split()) + len(sentence.split()) <= max_chunk_words:
            current_chunk += " " + sentence
        else:
            chunks.append(current_chunk.strip())
            current_chunk = sentence
    if current_chunk:
        chunks.append(current_chunk.strip())

    all_summaries = []
    for chunk in chunks:
        try:
            summary = text_summary(chunk, max_length=10000, min_length=25, do_sample=False)
            all_summaries.append(summary[0]['summary_text'])
        except Exception as e:
            all_summaries.append(f"[Error summarizing chunk: {e}]")

    return "\n\n".join(all_summaries)

# Extracts the YouTube video ID from a standard or short URL
def extract_video_id(youtube_url):
    parsed_url = urlparse(youtube_url)
    if parsed_url.hostname == "youtu.be":
        return parsed_url.path[1:]
    elif parsed_url.hostname in ["www.youtube.com", "youtube.com"]:
        return parse_qs(parsed_url.query).get("v", [None])[0]
    return None

# Fetch transcript and return summary
def get_transcript(video_url):
    video_id = extract_video_id(video_url)
    if not video_id:
        return " Invalid YouTube URL format."

    try:
        transcript = YouTubeTranscriptApi.get_transcript(video_id)
        full_text = "\n".join([entry['text'] for entry in transcript])
        summary = summarize_text(full_text)
        return summary
    except TranscriptsDisabled:
        return " Transcript is disabled for this video."
    except NoTranscriptFound:
        return "No transcript found for this video."
    except Exception as e:
        return f" An error occurred: {e}"

# Gradio UI
gr.Interface(
    fn=get_transcript,
    inputs=gr.Textbox(lines=2, label="Enter YouTube URL"),
    outputs=gr.Textbox(label="Summarized Transcript"),
    title="📺 YouTube Video Summarizer",
    description="Paste a YouTube video link with captions enabled. This app fetches the transcript and summarizes it using a DistilBART model fine-tuned on SAMSum/CNN."
).launch(share=True)


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
Device set to use cpu


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://0d361da808c01c9a4a.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
